In [1]:
%pip install keplergl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pydeck pandas h3 geopandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import geopandas as gpd
from keplergl import KeplerGl
import pandas as pd

In [8]:
import json

def load_config(file_path):
    try:
        with open(file_path, 'r') as file:
            config = json.load(file)

        # Odczytanie zmiennych z pliku konfiguracyjnego
        transmission_rate = config["SEIR_parameters"]["transmission_rate"]
        incubation_rate = config["SEIR_parameters"]["incubation_rate"]
        recovery_rate = config["SEIR_parameters"]["recovery_rate"]

        vaccination_rate = config["vaccination"]["vaccination_rate"]
        vaccination_enabled = config["vaccination"]["vaccination_enabled"]

        lockdown_threshold = config["lockdown"]["lockdown_threshold"]
        lockdown_time = config["lockdown"]["lockdown_time"]

        flights_enabled = config["flights_enabled"]

        initial_infected_population_h3 = config["initial_infected_population_h3"]

        # Zwrócenie wszystkich ustawień jako słownik
        return {
            "transmission_rate": transmission_rate,
            "incubation_rate": incubation_rate,
            "recovery_rate": recovery_rate,
            "vaccination_rate": vaccination_rate,
            "vaccination_enabled": vaccination_enabled,
            "lockdown_threshold": lockdown_threshold,
            "lockdown_time": lockdown_time,
            "flights_enabled": flights_enabled,
            "initial_infected_population_h3": initial_infected_population_h3
        }
    except (FileNotFoundError, KeyError, json.JSONDecodeError) as e:
        print(f"Error loading configuration: {e}")
        return None


In [9]:
config_path = "config.json"
settings = load_config(config_path)

if settings:
    print("Ustawienia wczytane poprawnie:")
    for key, value in settings.items():
        print(f"{key}: {value}")

Ustawienia wczytane poprawnie:
transmission_rate: 2
incubation_rate: 1
recovery_rate: 0.05
vaccination_rate: 0.0002
vaccination_enabled: False
lockdown_threshold: 0.4
lockdown_time: 30
flights_enabled: True
initial_infected_population_h3: {'846026bffffffff': 0.5, '841e25bffffffff': 0.5}


In [10]:
class Disease:
    def __init__(self):
        self.transmission_rate = settings.get("transmission_rate", 0.2)
        self.incubation_rate = settings.get("incubation_rate", 0.3)
        self.recovery_rate = settings.get("recovery_rate", 0.05)
        self.mortality_rate = 0

    def seir_rates(self):
        return self.transmission_rate, self.incubation_rate, self.recovery_rate, self.mortality_rate


In [14]:
import random


class PandemicSpreadSimulation:
    def __init__(self):
        self.vaccination_rate = settings.get("vaccination_rate", 0.01)
        self.vaccination_enabled = settings.get("vaccination_enabled", 0.2)
        self.lockdown_threshold = settings.get("lockdown_threshold", 0.2)
        self.lockdown_time = settings.get("lockdown_time", 30)
        self.flights_enabled = settings.get("flights_enabled", True)
        self.initial_infected_population_h3 = settings.get("initial_infected_population_h3", {})

        gdf = gpd.read_file('kontur_population_20231101_r4.gpkg')

        gdf = gdf[gdf['population'] > 50]

        # Upewnij się, że dane są w odpowiednim układzie współrzędnych (WGS84)
        gdf = gdf.to_crs(epsg=4326)

        # Dodaj pola SEIR do danych
        gdf['S'] = gdf['population']
        gdf['E'] = 0
        gdf['I'] = 0
        gdf['R'] = 0
        gdf['V'] = 0
        gdf['airport'] = False
        gdf['masks'] = False
        gdf['lockdown'] = 0
        gdf['vaccinations'] = False

        number_of_airports = 2000
        airport_indices = gdf.sample(n=number_of_airports, random_state=42).index
        gdf.loc[airport_indices, 'airport'] = True

        for hex_id, infection_progress in self.initial_infected_population_h3.items():
            target_hex = gdf[gdf['h3'] == hex_id]
            if not target_hex.empty:
                initial_infection = int(
                    target_hex['population'].values[0] * infection_progress
                )
                gdf.loc[gdf['h3'] == hex_id, 'I'] = initial_infection
                gdf.loc[gdf['h3'] == hex_id, 'S'] -= initial_infection
            else:
                print(f"No target hex found for h3 id {hex_id}")

        self.gdf = gdf
        self.disease = Disease()

    def infect_neighbors(self):
        for index, row in self.gdf.iterrows():
            infected = row['I']
            if infected == 0:
                continue

            neighbors = self.gdf[self.gdf.geometry.touches(row.geometry)]

            for neighbor_index, neighbor in neighbors.iterrows():
                neighbor_S = neighbor['S']

                transmission_probability = 1 - ((1 - infected / row['population']) ** 5)

                is_transmitted = random.random() < transmission_probability
                if is_transmitted:
                    new_exposed = random.randint(1, 10)  # Zakładamy losową liczbę nowych narażonych
                    actual_exposed = min(new_exposed, neighbor_S)  # Nie więcej niż dostępni w S

                    self.gdf.at[neighbor_index, 'S'] -= actual_exposed
                    self.gdf.at[neighbor_index, 'E'] += actual_exposed

        print("Neighbor infections processed.")

    # update simulation state by a day
    def update_simulation(self):
        self.infect_neighbors()

        if self.flights_enabled:
            self.simulate_flight()

        transmission_rate, incubation_rate, recovery_rate, mortality_rate = self.disease.seir_rates()

        for index, row in self.gdf.iterrows():
            S = row['S']
            E = row['E']
            I = row['I']
            R = row['R']
            population = row['population']
            lockdown_days = row['lockdown']

            # Decrease lockdown days if still active
            if lockdown_days > 0:
                self.gdf.loc[index, 'lockdown'] = lockdown_days - 1

            # Calculate new exposed, infected, recovered, and dead
            new_exposed = int(transmission_rate * S * I / population)
            new_infected = int(incubation_rate * E)
            new_recovered = int(recovery_rate * I)
            new_dead = int(mortality_rate * I)

            # Update SEIR values (ensuring no negative values)
            new_S = max(S - new_exposed, 0)
            new_E = max(E + new_exposed - new_infected, 0)
            new_I = max(I + new_infected - new_recovered - new_dead, 0)
            new_R = max(R + new_recovered, 0)

            if self.vaccination_enabled:
                new_V = new_S * self.vaccination_rate
                new_S = max(S - new_V, 0)
                self.gdf.loc[index, 'V'] = new_V

            # Update the values in the geodataframe using loc[]
            self.gdf.loc[index, 'S'] = new_S
            self.gdf.loc[index, 'E'] = new_E
            self.gdf.loc[index, 'I'] = new_I
            self.gdf.loc[index, 'R'] = new_R

    def simulate_flight(self):
        airports = self.gdf[(self.gdf['lockdown'] == False) & (self.gdf['airport'] == True)]

        for _, origin in airports.iterrows():
            if origin['I'] < 200000:
                continue

            sampled_airports = airports.sample(1)
            destination = sampled_airports.iloc[0]

            passengers = random.randint(100, 300)

            infected_probability = 1 - (1 - origin['I'] / origin['population'])

            print(infected_probability)

            is_infected = random.random() < infected_probability

            if is_infected:
                destination_S = self.gdf.at[destination.name, 'S']
                new_exposed = min(passengers, destination_S)  # Nie więcej niż S

                self.gdf.at[destination.name, 'S'] -= new_exposed
                self.gdf.at[destination.name, 'E'] += new_exposed

                print(f"Flight from {origin['h3']} to {destination['h3']}")
                print(f"Passengers: {passengers}")


In [15]:
import os
from datetime import datetime

simulation = PandemicSpreadSimulation()

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
folder_name = f"pandemic_spread_{current_time}"
os.makedirs(folder_name, exist_ok=True)

initial_state = simulation.gdf.copy()
initial_state.to_file(os.path.join(folder_name, "day_0.geojson"), driver="GeoJSON")

num_days = 20

print(f"\nRozpoczynam symulację dla {num_days - 1} kolejnych dni...")
for day in range(num_days - 1):
    print(f"\nDzień {day + 1}:")
    simulation.update_simulation()
    current_state = simulation.gdf.copy()
    print(f"- Liczba zarażonych (suma I): {current_state['I'].sum()}")
    print(f"- Liczba podatnych (suma S): {current_state['S'].sum()}")

    current_state.to_file(os.path.join(folder_name, f"day_{day + 1}.geojson"), driver="GeoJSON")

print("Koniec symulacji")


Rozpoczynam symulację dla 19 kolejnych dni...

Dzień 1:
Neighbor infections processed.
- Liczba zarażonych (suma I): 583927
- Liczba podatnych (suma S): 8030548633.0

Dzień 2:
Neighbor infections processed.
- Liczba zarażonych (suma I): 1169393
- Liczba podatnych (suma S): 8030548516.0

Dzień 3:
Neighbor infections processed.
- Liczba zarażonych (suma I): 1111037
- Liczba podatnych (suma S): 8030548319.0

Dzień 4:
Neighbor infections processed.
- Liczba zarażonych (suma I): 1055706
- Liczba podatnych (suma S): 8030547889.0

Dzień 5:
Neighbor infections processed.
- Liczba zarażonych (suma I): 1003346
- Liczba podatnych (suma S): 8030547052.0


DataSourceError: Failed to create GeoJSON datasource: pandemic_spread_2025-01-17_15-05-36\day_5.geojson: pandemic_spread_2025-01-17_15-05-36\day_5.geojson: No such file or directory

In [71]:
def create_map_with_simulation(simulation_states, filename='pandemic_simulation.html'):
    """
    Tworzy mapę z warstwami dla każdego dnia symulacji
    """
    print("Rozpoczynam tworzenie mapy...")

    try:
        # Wczytaj dane granic państw i miast
        print("Wczytuję dane granic państw...")
        countries = gpd.read_file("ne_110m_admin_0_countries.shp")
        countries.set_crs(epsg=4326, inplace=True)
        print("Wczytuję dane miast...")
        cities = gpd.read_file("ne_110m_populated_places.shp")
        cities.set_crs(epsg=4326, inplace=True)
        print("Dane bazowe wczytane pomyślnie")

        # Lista do przechowywania wszystkich stanów symulacji
        # simulation_states = []

        # Zapisz stan początkowy
        # print("Zapisuję stan początkowy symulacji...")
        initial_state = simulation_states[0]
        print(f"Stan początkowy - liczba rekordów: {len(initial_state)}")
        print(f"Kolumny w danych: {initial_state.columns.tolist()}")
        #simulation_states.append(initial_state)



        # Stwórz mapę
        print("\nTworzę obiekt mapy...")
        map_1 = KeplerGl(height=800)
        
        # Dodaj podstawowe dane
        print("Dodaję dane bazowe do mapy...")
        map_1.add_data(data=countries, name="countries")
        map_1.add_data(data=cities, name="cities")

        # Dodaj dane dla każdego dnia
        print("\nDodaję dane dla poszczególnych dni...")
        for day, state in enumerate(simulation_states):
            print(f"Dodaję dzień {day}...")
            print(f"- Kształt danych: {state.shape}")
            print(f"- Kolumna h3 obecna: {'h3' in state.columns}")
            print(f"- Kolumna I obecna: {'I' in state.columns}")
            map_1.add_data(data=state, name=f'pandemic_day_{day}')

        print("\nPrzygotowuję konfigurację warstw...")
        # Przygotuj warstwy dla każdego dnia
        layers = [
            # Warstwa granic państw
            {
                "id": "country_borders",
                "type": "geojson",
                "config": {
                    "dataId": "countries",
                    "label": "Countries",
                    "color": [200, 200, 200],
                    "columns": {"geojson": "geometry"},
                    "isVisible": True,
                    "visConfig": {
                        "opacity": 0.3,
                        "thickness": 1
                    }
                }
            },
            # Warstwa miast
            {
                "id": "cities_layer",
                "type": "geojson",
                "config": {
                    "dataId": "cities",
                    "label": "Cities",
                    "color": [255, 0, 0],
                    "columns": {"geojson": "geometry"},
                    "isVisible": True
                }
            }
        ]

        # Dodaj warstwę dla każdego dnia symulacji
        for day in range(len(simulation_states)):
            print(f"Dodaję konfigurację dla dnia {day}...")
            layer = {
                "id": f"infected_layer_day_{day}",
                "type": "hexagonId",
                "config": {
                    "dataId": f"pandemic_day_{day}",
                    "label": f"Infected Population - Day {day}",
                    "columns": {
                        "hex_id": "h3"
                    },
                    "isVisible": day == 0,
                    "visConfig": {
                                "coverage": 1,
                                "enable3d": True,
                                "sizeRange": [0, 500],
                                "coverageRange": [0, 1],
                                "elevationScale": 5,
                                "heightRange": [0, 500],
                                "colorScaleType": "linear",
                                "strokeColor": [255, 255, 255],
                                "strokeColorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": ["#FFF5F0", "#FEE0D2", "#FCBBA1", "#FC9272", "#FB6A4A", "#EF3B2C", "#CB181D", "#A50F15", "#67000D"]
                                }
                            }
                        },
                        "visualChannels": {
                            "colorField": {
                                "name": "I",
                                "type": "real",
                                "scale": "linear",
                                "domain": [0, 530900]  # Min i max wartości
                            },
                            "colorScale": "quantize",
                            "heightField": {
                                "name": "I",
                                "type": "real",
                                "scale": "linear",
                                "domain": [0, 530900]  # Min i max wartości
                            },
                            "sizeField": {
                                "name": "population",
                                "type": "real"
                            },
                            "sizeScale": "linear",
                            "heightScale": "linear"
                        }
            }
            
            
            layers.append(layer)

        # Konfiguracja mapy
        print("Tworzę końcową konfigurację mapy...")
        config = {
            "version": "v1",
            "config": {
                "visState": {
                    "filters": [],
                    "layers": layers
                },
                "mapState": {
                    "bearing": 0,
                    "latitude": 50,
                    "longitude": 20,
                    "pitch": 45,
                    "zoom": 4,
                    "dragRotate": True
                },
                "mapStyle": {
                    "styleType": "dark",
                    "topLayerGroups": {},
                    "visibleLayerGroups": {
                        "label": True,
                        "road": True,
                        "border": True,
                        "building": True,
                        "water": True,
                        "land": True
                    }
                }
            }
        }

        print("\nStosuje konfigurację do mapy...")
        map_1.config = config

        print(f"\nZapisuję mapę do pliku {filename}...")
        map_1.save_to_html(file_name=filename)
        
        print("\nMapa została utworzona pomyślnie!")
        return map_1

    except Exception as e:
        print(f"\nWYSTĄPIŁ BŁĄD: {str(e)}")
        import traceback
        print("\nSzczegóły błędu:")
        print(traceback.format_exc())
        raise

# Przykład użycia:
try:
    print("\nUruchamiam tworzenie mapy z symulacją...")

    directory = 'pandemic_spread_2025-01-16_20-26-32'
    geojson_files = [f for f in os.listdir(directory) if f.endswith('.geojson')]

    for geojson_file in geojson_files:
        state_name = os.path.splitext(geojson_file)[0]
        state_path = os.path.join(directory, geojson_file)
        state_data = gpd.read_file(state_path)
        infected_count = state_data['I'].sum() if 'I' in state_data else 0
        print(f"- Plik: {geojson_file}, Liczba zarażonych: {infected_count}")
        html_filename = os.path.join(directory, f"{state_name}.html")
        create_map_with_simulation([state_data], html_filename)
except Exception as e:
    print(f"\nBłąd podczas wykonywania głównego kodu: {str(e)}")


Uruchamiam tworzenie mapy z symulacją...
- Plik: day_0.geojson, Liczba zarażonych: 614616
Rozpoczynam tworzenie mapy...
Wczytuję dane granic państw...
Wczytuję dane miast...
Dane bazowe wczytane pomyślnie
Stan początkowy - liczba rekordów: 60790
Kolumny w danych: ['h3', 'population', 'S', 'E', 'I', 'R', 'V', 'airport', 'masks', 'lockdown', 'vaccinations', 'geometry']

Tworzę obiekt mapy...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Dodaję dane bazowe do mapy...

Dodaję dane dla poszczególnych dni...
Dodaję dzień 0...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True

Przygotowuję konfigurację warstw...
Dodaję konfigurację dla dnia 0...
Tworzę końcową konfigurację mapy...

Stosuje konfigurację do mapy...

Zapisuję mapę do pliku pandemic_spread_2025-01-16_20-26-32\day_0.html...
Map saved to pandemic_spread_2025-01-16_20-26-32\day_0.html!

Mapa została utworzona pomyślnie!
- Plik: day_1.geojson, Liczba zarażonych: 584217
Rozpoczynam tworzenie 